In [1]:
#Code to scrape only production budget data from The Numbers

In [2]:
#Required imports
import re
import pandas as pd
import requests
import numpy as np
import dateutil.parser
from bs4 import BeautifulSoup
from collections import defaultdict

In [32]:
#Code to scrape movie name, production date and release date 
movies=defaultdict(list)

def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

def money_to_int(moneystring):
    if ' million' in moneystring:
        moneystring = moneystring.replace('$', '').replace(' million', '')
        moneystring=int(moneystring)*1000000
        return int(moneystring)
    if '$' in moneystring: 
        moneystring = moneystring.replace('$', '').replace(',', '')
        return int(moneystring)        


url = 'http://www.the-numbers.com/movie/budgets/all'
response = requests.get(url)
response.status_code
page = response.text
soup = BeautifulSoup(page,"lxml")
table = soup.find("table")
rows=[row for row in table.find_all('tr')]

for row in rows:
    items=row.find_all('td')
    if len(items)>0:
        title=items[1].find('a')['href']
        movie=items[2].text
        date = dateutil.parser.parse(items[1].text)
        release_date = to_date(items[1].text)
        production_budget = money_to_int(items[3].text)
      
    
        movies[movie] = [release_date,production_budget]

len(movies.items())

5218

In [33]:
movies_df = pd.DataFrame.from_dict(movies, orient = 'index')

In [34]:
movies_df.head()

,0,1
Men with Brooms,2002-03-08,7500000
Dum Maaro Dum,2011-04-22,5200000
Vampires Suck,2010-08-18,20000000
The Road,2009-11-25,25000000
Yes Man,2008-12-19,50000000


In [35]:
movies_df = movies_df.reset_index()

In [36]:
movies_df.columns = ['movie_name', 'release_date', 'production_budget']
movies_df.columns

Index([u'movie_name', u'release_date', u'production_budget'], dtype='object')

In [37]:
movies_df = movies_df.fillna('')

In [38]:
movies_df.head()

,movie_name,release_date,production_budget
0,Men with Brooms,2002-03-08,7500000
1,Dum Maaro Dum,2011-04-22,5200000
2,Vampires Suck,2010-08-18,20000000
3,The Road,2009-11-25,25000000
4,Yes Man,2008-12-19,50000000


In [41]:
import sys;
reload(sys);
sys.setdefaultencoding("utf8")

In [42]:
movies_df.to_csv("numbersbudgets.csv")

In [44]:
less numbersbudgets.csv